# Performance investigation: Pyodide

References: 
* [https://gist.github.com/rth/c71fe792eb56fb271317e35e08576c7a#results]()
* [https://github.com/pyodide/pyodide/tree/main/benchmark/benchmarks]()

![](https://pbs.twimg.com/media/FDum1WfUUAADPMx?format=png&name=medium)

[_Norman-Nielsen Group_](https://www.nngroup.com/articles/response-times-3-important-limits/)

In [ ]:
import numpy as np
from time import time

N = 1000
X = np.random.RandomState(0).rand(N, N)
t0 = time()
X.dot(X)
print(f'Wall time: {time() - t0:.2f} s')

Wall time: 2.48 s


In [ ]:
import numpy as np


def allpairs_distances(A, B):
    """This returns the euclidean distances squared
    dist2(x, y) = dot(x, x) - 2 * dot(x, y) + dot(y, y)
    """
    A2 = np.einsum("ij,ij->i", A, A)
    B2 = np.einsum("ij,ij->i", B, B)
    return A2[:, None] + B2[None, :] - 2 * np.dot(A, B.T)

In [ ]:
import numpy as np


def allpairs_distances_loops(X, Y):
    result = np.zeros((X.shape[0], Y.shape[0]), X.dtype)
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            result[i, j] = np.sum((X[i, :] - Y[j, :]) ** 2)
    return result

In [ ]:
import numpy as np


def arc_distance(theta_1, phi_1, theta_2, phi_2):
    """
    Calculates the pairwise arc distance between all points in vector a and b.
    """
    temp = (
        np.sin((theta_2 - theta_1) / 2) ** 2
        + np.cos(theta_1) * np.cos(theta_2) * np.sin((phi_2 - phi_1) / 2) ** 2
    )
    distance_matrix = 2 * (np.arctan2(np.sqrt(temp), np.sqrt(1 - temp)))
    return distance_matrix

In [ ]:
import numpy as np


def check_mask(db, mask=[1, 0, 1]):
    out = np.zeros(db.shape[0], dtype=bool)
    for idx, line in enumerate(db):
        target, vector = line[0], line[1:]
        if (mask == np.bitwise_and(mask, vector)).all():
            if target == 1:
                out[idx] = 1
    return out

In [ ]:
import numpy as np


def create_grid(x):
    N = x.shape[0]
    z = np.zeros((N, N, 3))
    z[:, :, 0] = x.reshape(-1, 1)
    z[:, :, 1] = x
    fast_grid = z.reshape(N * N, 3)
    return fast_grid

In [ ]:
def cronbach(itemscores):
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)
    return nitems / (nitems - 1) * (1 - itemvars.sum() / tscores.var(ddof=1))

In [ ]:
def diffusion(u, tempU, iterNum):
    """
    Apply Numpy matrix for the Forward-Euler Approximation
    """
    mu = 0.1

    for n in range(iterNum):
        tempU[1:-1, 1:-1] = u[1:-1, 1:-1] + mu * (
            u[2:, 1:-1]
            - 2 * u[1:-1, 1:-1]
            + u[0:-2, 1:-1]
            + u[1:-1, 2:]
            - 2 * u[1:-1, 1:-1]
            + u[1:-1, 0:-2]
        )
        u[:, :] = tempU[:, :]
        tempU[:, :] = 0.0

In [ ]:
import numpy as np


def laplacian(grid):
    return (
        np.roll(grid, +1, 0)
        + np.roll(grid, -1, 0)
        + np.roll(grid, +1, 1)
        + np.roll(grid, -1, 1)
        - 4 * grid
    )


def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)

In [ ]:
import numpy as np


def fdtd(input_grid, steps):
    grid = input_grid.copy()
    old_grid = np.zeros_like(input_grid)
    previous_grid = np.zeros_like(input_grid)

    l_x = grid.shape[0]
    l_y = grid.shape[1]

    for i in range(steps):
        np.copyto(previous_grid, old_grid)
        np.copyto(old_grid, grid)

        for x in range(l_x):
            for y in range(l_y):
                grid[x, y] = 0.0
                if 0 < x + 1 < l_x:
                    grid[x, y] += old_grid[x + 1, y]
                if 0 < x - 1 < l_x:
                    grid[x, y] += old_grid[x - 1, y]
                if 0 < y + 1 < l_y:
                    grid[x, y] += old_grid[x, y + 1]
                if 0 < y - 1 < l_y:
                    grid[x, y] += old_grid[x, y - 1]

                grid[x, y] /= 2.0
                grid[x, y] -= previous_grid[x, y]

    return grid

In [ ]:
import numpy as np


def fft(x):
    return np.fft(x)

In [ ]:
import numpy as np


def grayscott(counts, Du, Dv, F, k):
    n = 100
    U = np.zeros((n + 2, n + 2), dtype=np.float32)
    V = np.zeros((n + 2, n + 2), dtype=np.float32)
    u, v = U[1:-1, 1:-1], V[1:-1, 1:-1]

    r = 20
    u[:] = 1.0
    U[n // 2 - r : n // 2 + r, n // 2 - r : n // 2 + r] = 0.50
    V[n // 2 - r : n // 2 + r, n // 2 - r : n // 2 + r] = 0.25
    u += 0.15 * np.random.random((n, n))
    v += 0.15 * np.random.random((n, n))

    for i in range(counts):
        Lu = (
            U[0:-2, 1:-1]
            + U[1:-1, 0:-2]
            - 4 * U[1:-1, 1:-1]
            + U[1:-1, 2:]
            + U[2:, 1:-1]
        )
        Lv = (
            V[0:-2, 1:-1]
            + V[1:-1, 0:-2]
            - 4 * V[1:-1, 1:-1]
            + V[1:-1, 2:]
            + V[2:, 1:-1]
        )
        uvv = u * v * v
        u += Du * Lu - uvv + F * (1 - u)
        v += Dv * Lv + uvv - (F + k) * v

    return V